Domain Adaptive Pretraining


In [6]:
import os
import json
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    HfArgumentParser
)
import torch

In [7]:
# Configuration
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
CORPUS_JSON = ["/content/cuad_sample_1.json"]
OUTPUT_DIR = "./legalbert-dapt"
MAX_LENGTH = 512
BATCH_SIZE = 16
GRAD_ACCUM_STEPS = 4
EPOCHS = 10
LEARNING_RATE = 1e-5
SEED = 1


In [8]:
# Load and flatten JSON
all_texts = []
for path in CORPUS_JSON:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for rec in data:
            txt = rec.get("text", "").strip()
            if len(txt) > 0:
                all_texts.append({"text": txt})
print(f"Total documents loaded: {len(all_texts)}")

# Create a HuggingFace Dataset
ds = Dataset.from_list(all_texts)

Total documents loaded: 5000


In [9]:
# Initialize tokenizer & model
print("Initializing tokenizer and model")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model_pretrain = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length"
    )

# Prepare tokenized dataset
print("Tokenizing dataset")
tokenized_ds = ds.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    load_from_cache_file=True,
)

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

Initializing tokenizer and model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Tokenizing dataset


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_steps=500,
    save_steps=2000,
    save_total_limit=2,
    seed=SEED,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=1
)

# Initialize Trainer
trainer = Trainer(
    model=model_pretrain,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Launch training
trainer.train()

<ipython-input-10-a8453e3a88d1>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hengc (hengc-boston-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.535700


TrainOutput(global_step=780, training_loss=0.5242459223820612, metrics={'train_runtime': 901.692, 'train_samples_per_second': 55.451, 'train_steps_per_second': 0.865, 'total_flos': 1.30065290293248e+16, 'train_loss': 0.5242459223820612, 'epoch': 9.881789137380192})

In [11]:
# Save the model
print(f"Saving model to {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)

Saving model to ./legalbert-dapt


###Supervised Fine Tuning:
Fine-tune the DAPT'd Legal-BERT model on the labeled seedset.

In [12]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset
import evaluate
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding

In [14]:
MODEL_PATH = "/content/legalbert-dapt"
LABELS_CSV = "/content/labeled_cuad.csv"
OUTPUT_DIR = "./legalbert-finetuned"
MAX_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 8e-6
SEED = 1
EPOCHS = 15

In [30]:
# Load labeled data
df = pd.read_csv(LABELS_CSV)
df = df.rename(columns={
    'predicted_label': 'label'
})


labels = ['Agency Agreements',
 'Confidentiality (NDA) Agreements',
 'Construction Contracts',
 'Employment Contracts',
 'Franchise Agreements',
 'Guarantee Contracts',
 'Indemnity Contracts',
 'Insurance Contracts',
 'Lease Agreements',
 'Licensing Agreements',
 'Loan Agreements',
 'Partnership Agreements',
 'Real Estate Contracts',
 'Sale of Goods',
 'Service Contracts',
 'Settlement Agreements']

df = df[df['label'].isin(labels)]

label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

df['label_id'] = df['label'].map(label2id)


# Train/val/test split
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label_id'], random_state=SEED)

datasets = {}
for split, split_df in [('train', train_df), ('validation', val_df), ('test', test_df)]:
    ds = Dataset.from_pandas(split_df[['snippet', 'label_id']].rename(columns={'label_id': 'labels'}))
    datasets[split] = ds

In [31]:
# Tokenizer & model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
).to(device)

def tokenize_batch(batch):
  texts = [str(t) for t in batch['snippet']]
  return tokenizer(texts, padding='max_length', truncation=True, max_length=MAX_LENGTH)

# Apply tokenization

for split in datasets:
  datasets[split] = datasets[split].map(tokenize_batch, batched=True)
  datasets[split] = datasets[split].remove_columns(['snippet'])
  datasets[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/legalbert-dapt and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2399 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [32]:
class_weights = torch.tensor(
    compute_class_weight('balanced', classes=np.arange(len(labels)), y=train_df['label_id']),
    dtype=torch.float
).to(device)

data_collator = DataCollatorWithPadding(tokenizer)
metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {'f1_macro': metric.compute(predictions=preds, references=labels, average='macro')['f1']}

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs['labels']
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs.logits
        loss = torch.nn.CrossEntropyLoss(weight=class_weights)(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [36]:
# Training arguments
training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        eval_strategy='epoch',
        save_strategy='epoch',
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model='f1_macro',
        greater_is_better=True,
        seed=SEED,
        fp16=torch.cuda.is_available(),
        report_to=["wandb"],
        run_name=wandb.run.name
    )

# Initialize Trainer
trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['validation'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

<ipython-input-36-05a38c4f1abb>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [37]:
# Train & evaluate
trainer.train()
metrics = trainer.evaluate(datasets['test'])
print(metrics)

Epoch,Training Loss,Validation Loss,F1 Macro
1,1.825900,1.686804,0.542140
2,1.058100,1.065270,0.656162
3,0.935200,0.901474,0.694448
4,0.533800,0.825516,0.737787
5,0.616200,0.828372,0.719306
6,0.438200,0.894649,0.701034
7,0.405800,0.872210,0.740057
8,0.284300,0.939339,0.739263
9,0.243700,1.001850,0.710599
10,0.163200,1.003667,0.728510


{'eval_loss': 0.8276713490486145, 'eval_f1_macro': 0.7902228544658294, 'eval_runtime': 1.1645, 'eval_samples_per_second': 257.612, 'eval_steps_per_second': 32.631, 'epoch': 15.0}


In [38]:
from sklearn.metrics import classification_report

predictions = trainer.predict(datasets['test'])
preds = np.argmax(predictions.predictions, axis=1)
class_ids = list(range(len(labels)))

report = classification_report(predictions.label_ids, preds, labels=class_ids, target_names=labels, digits=4)
print(report)

                                  precision    recall  f1-score   support

               Agency Agreements     0.3333    0.3333    0.3333         3
Confidentiality (NDA) Agreements     0.0000    0.0000    0.0000         7
          Construction Contracts     1.0000    1.0000    1.0000         1
            Employment Contracts     0.9429    0.7857    0.8571        42
            Franchise Agreements     1.0000    1.0000    1.0000         2
             Guarantee Contracts     0.8462    0.9167    0.8800        12
             Indemnity Contracts     1.0000    0.8462    0.9167        13
             Insurance Contracts     1.0000    1.0000    1.0000         3
                Lease Agreements     0.8095    1.0000    0.8947        17
            Licensing Agreements     0.6981    0.8605    0.7708        43
                 Loan Agreements     0.8462    0.9167    0.8800        24
          Partnership Agreements     0.8500    0.7727    0.8095        22
           Real Estate Contracts     